<a href="https://colab.research.google.com/github/ThuraAung1601/BHDD-using-streamlit/blob/main/CNN_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BHDD Dataset Preprocessing 
- Dataset Link : https://github.com/baseresearch/BHDD
( Download raw file directly and upload to your drive for further uses )

## Data Loading

In [5]:
import pickle
import numpy as np

#url_path : Path of dataset in your drive
url_path = "/content/gdrive/MyDrive/BHDD Experiments /data.pkl" 

dataset = []

with open(url_path,"rb") as file:
    dataset = pickle.load(file)
    
trainDataset = dataset["trainDataset"]
testDataset = dataset["testDataset"]

print("Train Size: {} Test Size: {}".format(len(trainDataset), len(testDataset)))
            

Train Size: 60000 Test Size: 27561


In [6]:
import urllib.request
import gzip
import pickle
import os
import numpy as np
from PIL import Image

In [7]:
numberOfClass = 10

trainX = [list(td["image"] / 255) for td in trainDataset]
trainY = np.eye(numberOfClass)[np.array([td["label"] for td in trainDataset]).reshape(-1)].tolist() #One-hot encode training labels
testX = [list(td["image"] / 255) for td in testDataset]
testY = np.eye(numberOfClass)[np.array([td["label"] for td in testDataset]).reshape(-1)].tolist() #One-hot encode test labels

In [8]:
print("TrainImageShape:" + str(np.shape(trainX)),
     "TrainLabelShape:" + str(np.shape(trainY)),
     "TestImageShape:" + str(np.shape(testX)),
     "TestLabelShape:" + str(np.shape(testY)))

TrainImageShape:(60000, 28, 28) TrainLabelShape:(60000, 10) TestImageShape:(27561, 28, 28) TestLabelShape:(27561, 10)


## Training BHDD using CNN


In [9]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [18]:
input_shape = (28,28,1)
num_classes = 10



model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 1600)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)               

In [19]:
# Scale images to the [0, 1] range
trainX = np.array(trainX)
testX = np.array(testX)
trainX = trainX.astype("float32") / 255
testX = testX.astype("float32") / 255

# Make sure images have shape (28, 28, 1)
trainX = np.expand_dims(trainX, -1)
testX = np.expand_dims(testX, -1)

print("x_train shape:", trainX.shape)
print(trainX.shape[0], "train samples")
print(testX.shape[0], "test samples")

x_train shape: (60000, 28, 28, 1)
60000 train samples
27561 test samples


In [20]:
trainY = np.array(trainY)
testY = np.array(testY)

In [ ]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy",keras.metrics.Precision(),keras.metrics.Recall()])

history = model.fit(trainX, trainY, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 43s 98ms/step - loss: 1.4221 - accuracy: 0.5887 - precision: 0.9493 - recall: 0.3150 - val_loss: 0.3158 - val_accuracy: 0.9237 - val_precision: 0.9542 - val_recall: 0.8810
Epoch 2/15
422/422 [==============================] - 40s 96ms/step - loss: 0.2681 - accuracy: 0.9257 - precision: 0.9459 - recall: 0.9040 - val_loss: 0.1884 - val_accuracy: 0.9470 - val_precision: 0.9604 - val_recall: 0.9387
Epoch 3/15
422/422 [==============================] - 41s 96ms/step - loss: 0.2051 - accuracy: 0.9416 - precision: 0.9550 - recall: 0.9303 - val_loss: 0.1582 - val_accuracy: 0.9565 - val_precision: 0.9666 - val_recall: 0.9460
Epoch 4/15
422/422 [==============================] - 41s 96ms/step - loss: 0.1770 - accuracy: 0.9499 - precision: 0.9601 - recall: 0.9409 - val_loss: 0.1323 - val_accuracy: 0.9640 - val_precision: 0.9726 - val_recall: 0.9578
Epoch 5/15
335/422 [======================>.......] - ETA: 8s - loss: 0.1579 - accuracy: 0.9562 

In [ ]:
print(history.history.keys())

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("model accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(['train','test'],loc='lower right')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(['train','test'],loc='upper right')
plt.show()

In [ ]:
score = model.evaluate(testX, testY, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
loss, accuracy, precision, recall = model.evaluate(testX, testY, verbose=0)
print('Loss', loss)
print('Accuracy', accuracy)
print('Precision', precision)
print('Recall', recall)

f1_score = (2* recall*precision)/(recall+precision)
print("f1_score", f1_score)

In [ ]:
from keras.models import load_model

model.save('./cnn.h5')  # creates a HDF5 file 'my_model.h5'